In [45]:
library(GenomicRanges)
library(data.table)

gtf="/sharedFolder/Data/refGene.gtf"
Database="/sharedFolder/Results/Hotspot/Database.csv"
gtfT=read.table(gtf,header=TRUE,sep="\t")
gtfG=GRanges(seqnames=as.matrix(gtfT[,1]),ranges=IRanges(start = as.numeric(gtfT[,2])-100,end=as.numeric(gtfT[,3])+100))
DatanabaseT=read.table(Database,header=TRUE,sep=",")
results="/sharedFolder/Results/Ligase4"

In [46]:
#FUNCTIONs
modify_vector <- function(vector) {
  modified_vector <- vector
  for (i in seq_along(vector)) {
    if (nchar(vector[i]) >= 10) {
      elements <- strsplit(vector[i], "_")[[1]]
      shortest_index <- which.min(nchar(elements))
      modified_vector[i] <- elements[shortest_index]
    }
  }
  return(modified_vector)
}

circoPlot=function(tempPath,increasedOnly,control,treat){
temp=read.table(tempPath,header=TRUE,row.names=1,sep=",")

if(increasedOnly){
temp=temp[which(temp[treat]-temp[control]>0),]
}    
data=read.table(paste("/sharedFolder/Results/Hotspot/Database_annotated.csv",sep=""),header=TRUE,sep=",")
dataList=c()
for(i in rownames(temp)){
if(!is.na(grep(i,data[,8])[1])){
    dataList=append(dataList,grep(i,data[,8])[1])
    }else{
    yo=strsplit(i,"_")[[1]]
    dataList = dataList=append(dataList,which(
  data$Chromosome == yo[1] &
  data$ChrStart == as.integer(yo[2]) &
  data$ChrEnd == as.integer(yo[3])
)[1])
    }
}
    
data2=data[dataList,]
tt=c("chr15","61816681","61822370","MYC")
names(tt)=names(data2[,c(1,2,3,8)])
label=rbind(data2[,c(1,2,3,8)],t(as.matrix(tt)))
condition=c("DMSO","Taze","Vale")
forever=matrix(ncol=7)
colnames(forever)=c("chr1","start1","end1","chr2","start2","end2","score")
label[,1]=gsub("chr","mm",label[,1])
nam=c(rownames(temp),"MYC")
nam[grep("chr",nam)]="NA"
label[,4]=modify_vector(nam)

write.table(label,paste(results,"/Ligase4_label.txt",sep=""),col.names=FALSE,row.names=FALSE,sep="\t",quote=FALSE)
test=matrix(ncol=7)
colnames(test)=c("chr1","start1","end1","chr2","start2","end2","score")
extra_info <- cbind(rep("chr15", nrow(data2)), rep("61816681", nrow(data2)), rep("61822370", nrow(data2)))
tt=cbind(extra_info,data2[,c(1,2,3)],paste("score=",(temp[,treat]+1)/(temp[,control]+1),sep=""))
    colnames(tt)=c("chr1","start1","end1","chr2","start2","end2","score")
    forever=rbind(forever,tt)
    test=rbind(forever,tt)
    test=test[-1,]
    
    test[,1]=gsub("chr","mm",test[,1])
    test[,4]=gsub("chr","mm",test[,4])
    write.table(test,paste(results,"/Ligase4_link.txt",sep=""),col.names=FALSE,row.names=FALSE,sep="\t",quote=FALSE)


forever=forever[-1,]

# Sostituisci "chr" con "mm" nelle colonne "chr1" e "chr2"

forever[,1]=gsub("chr","mm",forever[,1])
forever[,4]=gsub("chr","mm",forever[,4])

write.table(forever,paste(results,"/Ligase4_link.txt",sep=""),col.names=FALSE,row.names=FALSE,sep="\t",quote=FALSE)
setwd(results)
system("/home/circos-0.69-9/bin/circos -conf /sharedFolder/Circos_conf/yo.conf")
system(paste("mv /sharedFolder/Results/Ligase4/Ligase4.png ",tools::file_path_sans_ext(tempPath),".png",sep="")) 
system(paste("mv /sharedFolder/Results/Ligase4/Ligase4.svg ",tools::file_path_sans_ext(tempPath),".svg",sep=""))    

    system("/home/circos-0.69-9/bin/circos -conf /sharedFolder/Circos_conf/yo_NOTHICK.conf")
system(paste("mv /sharedFolder/Results/Ligase4/Ligase4.png ",tools::file_path_sans_ext(tempPath),"_noThick.png",sep="")) 
system(paste("mv /sharedFolder/Results/Ligase4/Ligase4.svg ",tools::file_path_sans_ext(tempPath),"_noThick.svg",sep=""))   
}

In [47]:
circoPlot("/sharedFolder/Results/Idelalisib/RPMCOmparison/DMSOrpm_FAIR_ANNOTATED.csv",FALSE,1,3)
circoPlot("/sharedFolder/Results/Idelalisib/RPMCOmparison/Tazerpm_FAIR_ANNOTATED.csv",FALSE,1,3)
circoPlot("/sharedFolder/Results/Idelalisib/RPMCOmparison/Valerpm_FAIR_ANNOTATED.csv",FALSE,1,3)
circoPlot("/sharedFolder/Results/Ligase4/rpmPlot/filtered/filtered_Lig_DMSOTaze_PlusAIDKO.csv",TRUE,1,2)
circoPlot("/sharedFolder/Results/Ligase4/rpmPlot/filtered/filtered_Lig_DMSOVale_PlusAIDKO.csv",TRUE,1,2)

